# Weather Prediction with Giry Monad - mULLER Paper Implementation

This notebook implements the weather prediction example from the mULLER paper, demonstrating proper monadic composition for neuro-symbolic reasoning.

## Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from scipy.stats import norm

from muller.monad.giry import (
    GiryMonad,
    binomial,
    fromDensityFunction,
    integrate
)

print("✅ All imports successful!")
print(f"NumPy version: {np.__version__}")

✅ All imports successful!
NumPy version: 2.3.2


## Neural Network Predictors

These functions simulate neural networks that output parameters for probability distributions.


In [2]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

def humid_detector(world_data):
    """
    Simulates a humidity detector that outputs parameters for a Bernoulli distribution.
    
    Args:
        world_data: Input data representing world state
        
    Returns:
        float: probability parameter p for Bernoulli(p) distribution
    """
    # Simulate neural network output - deterministic given world_data
    hash_val = hash(str(world_data)) % 1000000
    random.seed(hash_val)
    
    # Generate a probability between 0.1 and 0.9
    p = 0.1 + 0.8 * random.random()
    return p

def temperature_predictor(world_data):
    """
    Simulates a temperature predictor that outputs parameters for a Normal distribution.
    
    Args:
        world_data: Input data representing world state
        
    Returns:
        tuple: (mu, sigma) parameters for Normal(mu, sigma^2) distribution
    """
    # Simulate neural network output
    hash_val = hash(str(world_data)) % 1000000
    random.seed(hash_val + 12345)  # Different seed for temperature
    
    # Generate realistic temperature parameters
    mu = -10 + 50 * random.random()  # Temperature mean between -10 and 40°C
    sigma = 1 + 9 * random.random()  # Standard deviation between 1 and 10°C
    
    return mu, sigma

# Test the predictors
print("Testing Neural Network Predictors:")
print("=" * 40)

world_examples = ["sunny_day", "cloudy_day", "winter_scene"]

for world in world_examples:
    p = humid_detector(world)
    mu, sigma = temperature_predictor(world)
    
    print(f"World: {world}")
    print(f"  Humidity prob: {p:.3f}")
    print(f"  Temperature: N({mu:.2f}, {sigma:.2f}²)")
    print()


Testing Neural Network Predictors:
World: sunny_day
  Humidity prob: 0.253
  Temperature: N(16.16, 3.19²)

World: cloudy_day
  Humidity prob: 0.763
  Temperature: N(20.50, 2.94²)

World: winter_scene
  Humidity prob: 0.573
  Temperature: N(18.28, 5.96²)



## Proper Monadic Implementation

**The key insight**: The Giry monad's `bind` operation automatically handles the probabilistic integration.

#### Proper Monadic Approach ✅  
```python
# This IS the monadic way:
result = h_dist.bind(lambda h: 
           t_dist.bind(lambda t: 
             GiryMonad.unit(condition(h, t))))
```

The `bind` operations automatically:
1. **Sample from the first distribution** (humidity)
2. **For each sample, run the inner computation** (temperature)  
3. **Integrate over all possibilities** using measure theory
4. **Return the combined result** as a new measure


In [3]:
def is_good_weather(humidity, temperature):
    """
    Weather condition from the mULLER paper:
    (h = 1 ∧ t < 0) ∨ (h = 0 ∧ t > 15)
    
    Args:
        humidity: 0 or 1 (low or high humidity)
        temperature: temperature value in Celsius
        
    Returns:
        bool: True if weather is considered "good"
    """
    return (humidity == 1 and temperature < 0) or (humidity == 0 and temperature > 15)

def good_weather_query(world):
    """
    Implements the good_weather query using clean monadic composition.
    
    From the mULLER paper:
    h := bernoulli(humid_detector(World))
        t := normal(temperature_predictor(World))
            (h = 1 ∧ t < 0) ∨ (h = 0 ∧ t > 15)
    """
    humidity_p = humid_detector(world)
    temp_mu, temp_sigma = temperature_predictor(world)
    
    # Create distributions
    humidity_dist = binomial(1, humidity_p)
    temperature_dist = fromDensityFunction(
        lambda x: norm.pdf(x, temp_mu, temp_sigma)
    )
    
    # The monadic computation: humidity >>= (λh -> temperature >>= (λt -> unit(condition(h,t))))
    return humidity_dist.bind(
        lambda h: temperature_dist.bind(
            lambda t: GiryMonad.unit(
                1.0 if is_good_weather(h, t) else 0.0
            )
        )
    )


## Test the Monadic Implementation


In [4]:
# Test the monadic weather prediction
print("🔍 MONADIC WEATHER PREDICTION")
print("=" * 50)

test_worlds = ["beach_day", "mountain_winter", "desert_noon"]

print("The monadic approach:")
print("  result = humidity_dist.bind(λh -> temperature_dist.bind(λt -> unit(condition(h,t))))")
print("This automatically handles all the probabilistic integration!\n")

results = {}
for world in test_worlds:
    print(f"{'='*15} WORLD: {world} {'='*15}")
    
    # Test the clean monadic version 
    weather_dist = good_weather_query(world)
    prob_good_weather = integrate(lambda x: x, weather_dist.value)
    results[world] = prob_good_weather
    
    # Show what's happening
    humidity_p = humid_detector(world)
    temp_mu, temp_sigma = temperature_predictor(world)
    print(f"Neural predictions: h~Bernoulli({humidity_p:.3f}), t~N({temp_mu:.1f},{temp_sigma:.1f}²)")
    print(f"P(good_weather) = {prob_good_weather:.6f}")
    
    # Show the theoretical calculation for comparison
    prob_temp_less_0 = norm.cdf((0 - temp_mu) / temp_sigma)
    prob_temp_greater_15 = 1 - norm.cdf((15 - temp_mu) / temp_sigma)
    theoretical = humidity_p * prob_temp_less_0 + (1 - humidity_p) * prob_temp_greater_15
    
    print(f"Theoretical:    {theoretical:.6f}")
    print(f"Difference:     {abs(prob_good_weather - theoretical):.2e}")
    print()

print("Perfect agreement! The Giry monad handles the complex integration automatically!")


🔍 MONADIC WEATHER PREDICTION
The monadic approach:
  result = humidity_dist.bind(λh -> temperature_dist.bind(λt -> unit(condition(h,t))))
This automatically handles all the probabilistic integration!

=============== WORLD: beach_day ===============
Neural predictions: h~Bernoulli(0.149), t~N(26.8,4.5²)
P(good_weather) = 0.847301
Theoretical:    0.847301
Difference:     2.00e-15

=============== WORLD: mountain_winter ===============
Neural predictions: h~Bernoulli(0.766), t~N(14.5,6.1²)
P(good_weather) = 0.116132
Theoretical:    0.116132
Difference:     2.64e-16

=============== WORLD: desert_noon ===============
Neural predictions: h~Bernoulli(0.445), t~N(-9.0,1.8²)
P(good_weather) = 0.445488
Theoretical:    0.445488
Difference:     3.11e-14

Perfect agreement! The Giry monad handles the complex integration automatically!


## Summary: Monadic Weather Prediction

This focused implementation demonstrates the key aspects of the mULLER framework:

### **What We've Implemented:**

1. **Neural Network Simulators**: `humid_detector` and `temperature_predictor` functions
2. **Proper Monadic Composition**: Using `bind` operations for sequential probabilistic computation
3. **Perfect Accuracy**: Monadic results match theory to machine precision

### **Key Monadic Insights:**

- **Sequential Composition**: `bind` chains computations where later steps depend on earlier results
- **Automatic Integration**: The monad handles measure-theoretic integration automatically  
- **Clean Code**: The monadic pattern produces readable, composable code

### **The Pattern:**

```python
result = humidity_dist.bind(
    lambda h: temperature_dist.bind(
        lambda t: GiryMonad.unit(condition(h, t))
    )
)
```

**This demonstrates that the Giry monad provides a practical, mathematically sound foundation for neuro-symbolic reasoning as described in the mULLER paper.**
